In [1]:
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from nltk.sentiment.vader import SentimentIntensityAnalyzer

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.metrics import plot_confusion_matrix, confusion_matrix
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.neighbors import KNeighborsClassifier

## Democrat Subreddit Collection

In [2]:
#data collection from Dems Subreddit
url= 'https://api.pushshift.io/reddit/search/submission'

#pull the fist 250 posts 
params1 = {
            'subreddit' : 'democrats',
            'size' : 250
        }
req = requests.get(url, params1)
dem_data = req.json()
tempdf= dem_data['data'] 
dems_df= pd.DataFrame(tempdf)

#using a for loop to pull ~10k posts in batches of 250 (limit set by reddit API)
for i in range(1,40):
    params2 = {
            'subreddit' : 'democrats',
            'size' : 250,
            'before': dems_df['created_utc'][-1:]
        }

    req2 = requests.get(url, params2)
    dem_data2 = req2.json()
    tempdf2= dem_data2['data']
    dems_df= dems_df.append(tempdf2)
    

C:\Users\dylan\AppData\Local\Temp\ipykernel_7224\2413850914.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dems_df= dems_df.append(tempdf2)
C:\Users\dylan\AppData\Local\Temp\ipykernel_7224\2413850914.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dems_df= dems_df.append(tempdf2)
C:\Users\dylan\AppData\Local\Temp\ipykernel_7224\2413850914.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dems_df= dems_df.append(tempdf2)
C:\Users\dylan\AppData\Local\Temp\ipykernel_7224\2413850914.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dems_df= dems_df.append(tempdf2)
C:\Users\dylan\AppData\Local\Temp\ipykernel_7224\2413850914.

In [4]:
dems_df.columns

Index(['all_awardings', 'allow_live_comments', 'author',
       'author_flair_css_class', 'author_flair_richtext', 'author_flair_text',
       'author_flair_type', 'author_fullname', 'author_is_blocked',
       'author_patreon_flair', 'author_premium', 'awarders', 'can_mod_post',
       'contest_mode', 'created_utc', 'domain', 'full_link', 'gildings', 'id',
       'is_created_from_ads_ui', 'is_crosspostable', 'is_meta',
       'is_original_content', 'is_reddit_media_domain', 'is_robot_indexable',
       'is_self', 'is_video', 'link_flair_background_color',
       'link_flair_richtext', 'link_flair_text', 'link_flair_text_color',
       'link_flair_type', 'locked', 'media_only', 'no_follow', 'num_comments',
       'num_crossposts', 'over_18', 'parent_whitelist_status', 'permalink',
       'pinned', 'post_hint', 'preview', 'pwls', 'removed_by_category',
       'retrieved_on', 'score', 'selftext', 'send_replies', 'spoiler',
       'stickied', 'subreddit', 'subreddit_id', 'subreddit_subscr

In [5]:
demdata= dems_df[['subreddit', 'title', 'selftext', 'author' ]]

In [6]:
demdata.head()

,subreddit,title,selftext,author
0,democrats,The Georgia Peach 🍑,,ImaginationFree6807
1,democrats,Hurricane Ian’s Toll Is Severe. Lack of Insura...,1. Vote for politicians who don’t believe in c...,TexasBogart
2,democrats,No audience allowed for Friday debate between ...,,gotostep2
3,democrats,The Georgia Peach 🍑 🤣🤣,,ImaginationFree6807
4,democrats,Truth,,Lali-confused


In [7]:
demdata['title'] = demdata['title'].str.lower()

C:\Users\dylan\AppData\Local\Temp\ipykernel_7224\317194077.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  demdata['title'] = demdata['title'].str.lower()


## Republican Subreddit Collection

In [8]:
#data collection from Republican Subreddit
url= 'https://api.pushshift.io/reddit/search/submission'

#pull the fist 250 posts 
params1 = {
            'subreddit' : 'Republican',
            'size' : 250
        }
req = requests.get(url, params1)
rep_data = req.json()
tempdf= rep_data['data'] 
rep_df= pd.DataFrame(tempdf)

#using a for loop to pull ~3k posts in batches of 250 (limit set by reddit API)
for i in range(1,40):
    params2 = {
            'subreddit' : 'Republican',
            'size' : 250,
            'before': rep_df['created_utc'][-1:]
        }

    req2 = requests.get(url, params2)
    rep_data2 = req2.json()
    tempdf2= rep_data2['data']
    rep_df= rep_df.append(tempdf2)

C:\Users\dylan\AppData\Local\Temp\ipykernel_7224\462446494.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rep_df= rep_df.append(tempdf2)
C:\Users\dylan\AppData\Local\Temp\ipykernel_7224\462446494.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rep_df= rep_df.append(tempdf2)
C:\Users\dylan\AppData\Local\Temp\ipykernel_7224\462446494.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rep_df= rep_df.append(tempdf2)
C:\Users\dylan\AppData\Local\Temp\ipykernel_7224\462446494.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rep_df= rep_df.append(tempdf2)
C:\Users\dylan\AppData\Local\Temp\ipykernel_7224\462446494.py:25: Future

In [10]:
repdata= rep_df[['subreddit', 'title', 'selftext', 'author' ]]

In [11]:
repdata['title'] = repdata['title'].str.lower()

C:\Users\dylan\AppData\Local\Temp\ipykernel_7224\666624353.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  repdata['title'] = repdata['title'].str.lower()


In [12]:
%store demdata
%store repdata

Stored 'demdata' (DataFrame)
Stored 'repdata' (DataFrame)
